In [1]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [ ]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [11]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,EGRESO
1,BENEFICIARIO RENOVANTE
2,DESERCION
3,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
4,GRATUIDAD
5,NUEVO BENEFICIARIO


In [ ]:
query_sum = """

SELECT id_2023 as id,
año_licitacion as año_solicitud,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado,
        ROUND(AVG(porcentaje_financiado),1) as porcentaje_financiado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_solicitud
"""

In [5]:
final_table_to_query = executes_query_duckdb(query_sum, cae)
final_table_to_query.to_parquet("../data/summary_table.parquet")

In [6]:
final_table_to_query

,id,año_pedido,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado
0,625.879,2013,F,Particular subvencionado,REGION DEL BIO-BIO,quintil 4,1,1,2,1,0,3474232.0
1,625.888,2013,F,Municipal,REGION DEL BIO-BIO,quintil 4,1,1,5,1,0,4938549.0
2,689.499,2013,F,Municipal,REGION DE COQUIMBO,quintil 3,1,1,3,1,0,3803563.0
3,691.213,2014,M,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,4,1,0,4703989.0
4,627.079,2011,F,Particular subvencionado,REGION DEL BIO-BIO,quintil 1 y 2,1,1,6,1,0,21730929.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,1.240.825,2024,F,Municipal DAEM,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,7392032.0
1290989,631.990,2024,F,Particular Subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,2661658.0
1290990,1.222.388,2024,M,Particular Subvencionado,REGION DE ÑUBLE,quintil 4,1,1,1,0,0,6246108.0
1290991,1.223.369,2024,F,Particular Pagado,REGION METROPOLITANA,quintil 5,1,1,1,0,0,8345310.0


In [17]:
h = executes_query_duckdb("select * from cae limit 100000", cae)
h

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,porcentaje_financiado,total_prestado_ajustado
0,667.946,2012,2012,F,Chilena,REGION DE VALPARAISO,quintil 4,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,2043680,2043680,Profesional con Licenciatura,Diurna,ESTADO,1.0,3449731.840
1,958.754,2017,2017,F,Chilena,REGION DE LA ARAUCANIA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,7,1,Particular subvencionado,3674581,3674581,Profesional con Licenciatura,Diurna,ESTADO,1.0,5250976.249
2,668.482,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 5,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Particular subvencionado,820429,820429,Profesional con Licenciatura,Diurna,ESTADO,1.0,1384884.152
3,676.859,2012,2012,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,3,1,Municipal,1113000,1113000,Técnico,Diurna,ESTADO,1.0,1878744.000
4,677.231,2012,2012,F,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Particular subvencionado,1523626,1523626,Profesional,Diurna,ESTADO,1.0,2571880.688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,398.205,2010,2010,F,Chilena,REGION DEL MAULE,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,5,1,Municipal,1340000,1340000,Profesional con Licenciatura,Diurna,SCOTIABANK,1.0,2398600.000
99996,283.103,2008,2008,M,Chilena,REGION DE VALPARAISO,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,4,1,Estudiante de Colegio o Liceo Particular subve...,1259168,1259168,Profesional,Diurna,SCOTIABANK,1.0,2287908.256
99997,543.917,2011,2017,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,5,6,Municipal,0,0,Profesional con Licenciatura,Diurna,ITAU,NaN,0.000
99998,143.034,2008,2010,F,Chilena,REGION DE COQUIMBO,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,3,3,Estudiante de Colegio o Liceo Fiscal o Municipal,490000,490000,Técnico,Diurna,ESTADO,1.0,877100.000
